# Exploratory Analysis of Currency Exchange Rates - USD vs Other Countries

In this Python notebook, we'll be investigating the strength of the US Dollar against 3 other countries over the last 7 election cyles (28 years) to understand how the exchange rates change before and after Election Day in a 30-day window split into two fifteen day periods. 

We'll explore the volatility of the currencies during this window. The currencies of our focused analysis are: Euro (EUR), Japananese Yen (JPY), and the Chinese Yuan (CNY). 

All of our data will be pulled from Federal Reserve at [https://www.federalreserve.gov/](https://www.federalreserve.gov/).

In [ ]:
# Install the required libraries
!pip install prophet

# Import Dependencies
import pandas as pd
from prophet import Prophet
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import calendar

%matplotlib inline

# Step 1: Data Prep

## Let's collect daily exchange rate data for the currencies - CNY, JPY and EUR
Source: [https://www.federalreserve.gov/releases/h10/hist/default.htm](https://www.federalreserve.gov/releases/h10/hist/default.htm)

Our method is to save the raw html file locally and use the read_html() function to read the tables. There will be an html file for each country in the 'Resources' folder. 

In [ ]:
# list of all countries by their currency abbreviation 
currencies = ['CNY', 'JPY', 'EUR']

# Past 8 election dates
election_dates = pd.to_datetime(['2000-11-07', '2004-11-03', '2008-11-04', '2012-11-06', '2016-11-08', '2020-11-03', '2024-11-05'])
print(election_dates)

In [495]:
# We need to get the exact dates of the windows we're analayzing (15 days before and after election day)
# Let's create an empty dictionary for election_windows
election_windows = {}

#calculate the total 30 days (15 before and 15 after) for each election date
for date in election_dates:
    start_date = date - pd.Timedelta(days=15)
    end_date = date + pd.Timedelta(days=15)
    election_windows[date.year] = (start_date, end_date)

In [ ]:
# Preview the election_windows, where the first key is the year, the first key-pair date is the 'before date' and the second key-pair is the 'after date.'
election_windows

In [ ]:
# HTML containing the currency rate data was downloaded and stored in the 'Resources' folder
# All url's stored as variables
china_url = 'Resources/FRB_daily_china.html'
japan_url = 'Resources/FRB_daily_japan.html'
euro_url = 'Resources/FRB_daily_euro.html'

# Read the html file into the tables variable, setting the Date column as the index
tables_china_daily_xchange = pd.read_html(china_url, index_col='Date', parse_dates=True)
tables_japan_daily_xchange = pd.read_html(japan_url, index_col='Date', parse_dates=True)
tables_euro_daily_xchange = pd.read_html(euro_url, index_col='Date', parse_dates=True)

# Let's check how many tables we extracted
print(f'There are {len(tables_china_daily_xchange)} table(s) extracted from the China exchanges rates url.')
print(f'There are {len(tables_japan_daily_xchange)} table(s) extracted from the Japan exchanges rates url.')
print(f'There are {len(tables_euro_daily_xchange)} table(s) extracted from the Euro exchanges rates url.')

# Create the data frames for all of our exchange rates 
df_china_daily = tables_china_daily_xchange[0]
df_japan_daily = tables_japan_daily_xchange[0]
df_euro_daily = tables_euro_daily_xchange[0]

# Preview our df's
display(df_china_daily)
display(df_japan_daily)
display(df_euro_daily)

In [ ]:
# Let's rename the 'Rate' column to their respective their country
df_china_daily.rename(columns={'Rate': 'CNY'}, inplace=True)
df_japan_daily.rename(columns={'Rate': 'JPY'}, inplace=True)
df_euro_daily.rename(columns={'Rate': 'EUR'}, inplace=True)

# Preview each dataframe to confirm the addition of the column
display(df_china_daily)
display(df_japan_daily)
display(df_euro_daily)

In [499]:
# 'ND' is a placeholder for where currency exchange data isn't provided
# On each dataframe, let's replace value 'ND' with NaN
df_china_daily = df_china_daily.replace('ND', np.nan)
df_japan_daily = df_japan_daily.replace('ND', np.nan)
df_euro_daily = df_euro_daily.replace('ND', np.nan)

In [ ]:
# Let's merge all three data frames based on the date index
df_currency_rates = pd.concat([df_china_daily, df_japan_daily, df_euro_daily], axis='columns', join='inner')
display(df_currency_rates)

In [ ]:
# Let's investigate all the data types of our newly merged df
df_currency_rates.info()

In [ ]:
# Let's convert all of our rate data to float type and verify 
df_currency_rates['CNY'] = pd.to_numeric(df_currency_rates['CNY'], errors='coerce')
df_currency_rates['JPY'] = pd.to_numeric(df_currency_rates['JPY'], errors='coerce')
df_currency_rates['EUR'] = pd.to_numeric(df_currency_rates['EUR'], errors='coerce')

df_currency_rates.dtypes

In [ ]:
# Check for null values
display(df_currency_rates.shape)
display(df_currency_rates.isnull().sum())

In [504]:
# 255/6535 is a small percentage of the data, so we can drop the rows with null values
df_currency_rates = df_currency_rates.dropna()

In [ ]:
# Verify 0 null values
df_currency_rates.isnull().sum()

In [506]:
# Let's get the data for the election windows for 2000
exchange_rates_2000_before = df_currency_rates[(df_currency_rates.index >= election_windows[2000][0]) & (df_currency_rates.index < election_dates[0])]
exchange_rates_2000_after = df_currency_rates[(df_currency_rates.index >= election_dates[0]) & (df_currency_rates.index <= election_windows[2000][1])]

# Let's get the data for the election windows for 2004
exchange_rates_2004_before = df_currency_rates[(df_currency_rates.index >= election_windows[2004][0]) & (df_currency_rates.index < election_dates[1])]
exchange_rates_2004_after = df_currency_rates[(df_currency_rates.index >= election_dates[1]) & (df_currency_rates.index <= election_windows[2004][1])]

# Let's get the data for the election windows for 2008
exchange_rates_2008_before = df_currency_rates[(df_currency_rates.index >= election_windows[2008][0]) & (df_currency_rates.index < election_dates[2])]
exchange_rates_2008_after = df_currency_rates[(df_currency_rates.index >= election_dates[2]) & (df_currency_rates.index <= election_windows[2008][1])]

# Let's get the data for the election windows for 2012
exchange_rates_2012_before = df_currency_rates[(df_currency_rates.index >= election_windows[2012][0]) & (df_currency_rates.index < election_dates[3])]
exchange_rates_2012_after = df_currency_rates[(df_currency_rates.index >= election_dates[3]) & (df_currency_rates.index <= election_windows[2012][1])]

# Let's get the data for the election windows for 2016
exchange_rates_2016_before = df_currency_rates[(df_currency_rates.index >= election_windows[2016][0]) & (df_currency_rates.index < election_dates[4])]
exchange_rates_2016_after = df_currency_rates[(df_currency_rates.index >= election_dates[4]) & (df_currency_rates.index <= election_windows[2016][1])]

# Let's get the data for the election windows for 2020
exchange_rates_2020_before = df_currency_rates[(df_currency_rates.index >= election_windows[2020][0]) & (df_currency_rates.index < election_dates[5])]
exchange_rates_2020_after = df_currency_rates[(df_currency_rates.index >= election_dates[5]) & (df_currency_rates.index <= election_windows[2020][1])]

# Let's get the data for the election windows for 2024
exchange_rates_2024_before = df_currency_rates[(df_currency_rates.index >= election_windows[2024][0]) & (df_currency_rates.index < election_dates[6])]
exchange_rates_2024_after = df_currency_rates[(df_currency_rates.index >= election_dates[6]) & (df_currency_rates.index <= election_windows[2024][1])]


In [ ]:
# Let's Preview some of the data to verify
display(exchange_rates_2000_before)
display(exchange_rates_2000_after)

display(exchange_rates_2024_before)
display(exchange_rates_2024_after)

In [508]:
# Combine both before and after data frames for each of the election years
exchange_rates_2020_combo = pd.concat([exchange_rates_2020_before, exchange_rates_2020_after])
exchange_rates_2004_combo = pd.concat([exchange_rates_2004_before, exchange_rates_2004_after])
exchange_rates_2008_combo = pd.concat([exchange_rates_2008_before, exchange_rates_2008_after])
exchange_rates_2012_combo = pd.concat([exchange_rates_2012_before, exchange_rates_2012_after])
exchange_rates_2016_combo = pd.concat([exchange_rates_2016_before, exchange_rates_2016_after])
exchange_rates_2020_combo = pd.concat([exchange_rates_2020_before, exchange_rates_2020_after])
exchange_rates_2024_combo = pd.concat([exchange_rates_2024_before, exchange_rates_2024_after])


In [509]:
# Let's show only day and month for each of the DFs
exchange_rates_2020_combo['DayMonth'] = exchange_rates_2020_combo.index.strftime('%d-%b')
exchange_rates_2004_combo['DayMonth'] = exchange_rates_2004_combo.index.strftime('%d-%b')
exchange_rates_2008_combo['DayMonth'] = exchange_rates_2008_combo.index.strftime('%d-%b')
exchange_rates_2012_combo['DayMonth'] = exchange_rates_2012_combo.index.strftime('%d-%b')
exchange_rates_2016_combo['DayMonth'] = exchange_rates_2016_combo.index.strftime('%d-%b')
exchange_rates_2020_combo['DayMonth'] = exchange_rates_2020_combo.index.strftime('%d-%b')
exchange_rates_2024_combo['DayMonth'] = exchange_rates_2024_combo.index.strftime('%d-%b')

In [510]:
# Ensure the DRs are sorted by DayMonth
exchange_rates_2000_combo = exchange_rates_2020_combo.sort_index()
exchange_rates_2004_combo = exchange_rates_2004_combo.sort_index()
exchange_rates_2008_combo = exchange_rates_2008_combo.sort_index()
exchange_rates_2012_combo = exchange_rates_2012_combo.sort_index()
exchange_rates_2016_combo = exchange_rates_2016_combo.sort_index()
exchange_rates_2020_combo = exchange_rates_2020_combo.sort_index()
exchange_rates_2024_combo = exchange_rates_2024_combo.sort_index()

# Step 2: Visualize Data

# CNY

In [ ]:
# USE THIS FOR FINAL PLOT
plt.clf()

# Create the figure
plt.figure(figsize=(18, 6))

# Plot each line using the combo dataframes and DayMonth column
plt.plot(range(len(exchange_rates_2000_combo)), exchange_rates_2000_combo['CNY'], label='2000', color='brown')
plt.plot(range(len(exchange_rates_2004_combo)), exchange_rates_2004_combo['CNY'], label='2004', color='green')
plt.plot(range(len(exchange_rates_2008_combo)), exchange_rates_2008_combo['CNY'], label='2008', color='blue')
plt.plot(range(len(exchange_rates_2012_combo)), exchange_rates_2012_combo['CNY'], label='2012', color='red')
plt.plot(range(len(exchange_rates_2016_combo)), exchange_rates_2016_combo['CNY'], label='2016', color='purple')
plt.plot(range(len(exchange_rates_2020_combo)), exchange_rates_2020_combo['CNY'], label='2020', color='orange')
plt.plot(range(len(exchange_rates_2024_combo)), exchange_rates_2024_combo['CNY'], label='2024', color='black')

# Customize the plot
plt.title('USD to CNY Exchange Rate Before and After Election Day')
plt.xlabel('Days')
plt.ylabel('Exchange Rate (USD/CNY)')

# Use the DayMonth from one of the combo dataframes for x-axis labels
# Show every 3rd label to avoid overcrowding
x_ticks = range(0, len(exchange_rates_2004_combo), 3)
plt.xticks(x_ticks, exchange_rates_2004_combo['DayMonth'].iloc[x_ticks], rotation=45)

# Add vertical line at election day (middle point)
middle_idx = len(exchange_rates_2004_combo) // 2
plt.axvline(x=middle_idx, color='gray', linestyle='--', alpha=0.5)

# Add grid for better readability
plt.grid(True, alpha=0.3)

# Add legend
plt.legend()

# Adjust layout to prevent label cutoff
plt.tight_layout()

# Display the chart
plt.show()

## EUR

In [ ]:
# USE THIS FOR FINAL PLOT
# AI help
plt.clf()

# Create the figure
plt.figure(figsize=(18, 6))

# Plot each line using the combo dataframes and DayMonth column
plt.plot(range(len(exchange_rates_2000_combo)), exchange_rates_2000_combo['EUR'], label='2000', color='brown')
plt.plot(range(len(exchange_rates_2004_combo)), exchange_rates_2004_combo['EUR'], label='2004', color='green')
plt.plot(range(len(exchange_rates_2008_combo)), exchange_rates_2008_combo['EUR'], label='2008', color='blue')
plt.plot(range(len(exchange_rates_2012_combo)), exchange_rates_2012_combo['EUR'], label='2012', color='red')
plt.plot(range(len(exchange_rates_2016_combo)), exchange_rates_2016_combo['EUR'], label='2016', color='purple')
plt.plot(range(len(exchange_rates_2020_combo)), exchange_rates_2020_combo['EUR'], label='2020', color='orange')
plt.plot(range(len(exchange_rates_2024_combo)), exchange_rates_2024_combo['EUR'], label='2024', color='black')

# Customize the plot
plt.title('USD to EUR Exchange Rate Before and After Election Day')
plt.xlabel('Days')
plt.ylabel('Exchange Rate (USD/EUR)')

# Use the DayMonth from one of the combo dataframes for x-axis labels
# Show every 3rd label to avoid overcrowding
x_ticks = range(0, len(exchange_rates_2004_combo), 3)
plt.xticks(x_ticks, exchange_rates_2004_combo['DayMonth'].iloc[x_ticks], rotation=45)

# Add vertical line at election day (middle point)
middle_idx = len(exchange_rates_2004_combo) // 2
plt.axvline(x=middle_idx, color='gray', linestyle='--', alpha=0.5)

# Add grid for better readability
plt.grid(True, alpha=0.3)

# Add legend
plt.legend()

# Adjust layout to prevent label cutoff
plt.tight_layout()

# Display the chart
plt.show()

## JPY

In [ ]:
# USE THIS FOR FINAL PLOT
# AI help
plt.clf()

# Create the figure
plt.figure(figsize=(18, 6))

# Plot each line using the combo dataframes and DayMonth column
plt.plot(range(len(exchange_rates_2000_combo)), exchange_rates_2000_combo['JPY'], label='2000', color='brown')
plt.plot(range(len(exchange_rates_2004_combo)), exchange_rates_2004_combo['JPY'], label='2004', color='green')
plt.plot(range(len(exchange_rates_2008_combo)), exchange_rates_2008_combo['JPY'], label='2008', color='blue')
plt.plot(range(len(exchange_rates_2012_combo)), exchange_rates_2012_combo['JPY'], label='2012', color='red')
plt.plot(range(len(exchange_rates_2016_combo)), exchange_rates_2016_combo['JPY'], label='2016', color='purple')
plt.plot(range(len(exchange_rates_2020_combo)), exchange_rates_2020_combo['JPY'], label='2020', color='orange')
plt.plot(range(len(exchange_rates_2024_combo)), exchange_rates_2024_combo['JPY'], label='2024', color='black')

# Customize the plot
plt.title('USD to JPY Exchange Rate Before and After Election Day')
plt.xlabel('Days')
plt.ylabel('Exchange Rate (USD/JPY)')

# Use the DayMonth from one of the combo dataframes for x-axis labels
# Show every 3rd label to avoid overcrowding
x_ticks = range(0, len(exchange_rates_2004_combo), 3)
plt.xticks(x_ticks, exchange_rates_2004_combo['DayMonth'].iloc[x_ticks], rotation=45)

# Add vertical line at election day (middle point)
middle_idx = len(exchange_rates_2004_combo) // 2
plt.axvline(x=middle_idx, color='gray', linestyle='--', alpha=0.5)

# Add grid for better readability
plt.grid(True, alpha=0.3)

# Add legend
plt.legend()

# Adjust layout to prevent label cutoff
plt.tight_layout()

# Display the chart
plt.show()

In [ ]:
# Using the exchange_rates_2004_combo dataframe, create a copy to another data frame and drop the DayMonth column
exchange_rates_2004_combo_copy = exchange_rates_2004_combo.copy()
exchange_rates_2004_combo_copy.drop(columns='DayMonth', inplace=True)

display(exchange_rates_2004_combo_copy)

In [ ]:
exchange_rates_2004_combo_copy.info()

In [ ]:
# Example of 30 day window for 2024
normalized_rates = exchange_rates_2004_combo_copy.apply(lambda x: (x - x.min()) / (x.max() - x.min()))

plt.clf()
plt.figure(figsize=(18, 6))
plt.plot(normalized_rates.index, normalized_rates['EUR'], label='EUR', color='blue')
plt.plot(normalized_rates.index, normalized_rates['JPY'], label='JPY', color='red')
plt.plot(normalized_rates.index, normalized_rates['CNY'], label='CNY', color='green')
plt.title('USD to Foreign Exchange Rate for 2024')
plt.xlabel('Months')
plt.ylabel('Exchange Rate')

plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
normalized_rates = df_currency_rates.apply(lambda x: (x - x.min()) / (x.max() - x.min()))

plt.clf()
plt.figure(figsize=(18, 6))
plt.plot(normalized_rates.index, normalized_rates['EUR'], label='EUR', color='blue')
plt.plot(normalized_rates.index, normalized_rates['JPY'], label='JPY', color='red')
plt.plot(normalized_rates.index, normalized_rates['CNY'], label='CNY', color='green')
plt.title('Normalized USD to Foreign Exchange Rate From 2004-2024')
plt.xlabel('Year')
plt.ylabel('Exchange Rate')

ax = plt.gca()  # Get the current axis
ax.xaxis.set_major_locator(mdates.YearLocator())  # Major ticks at the start of each month
#ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))  # Format as abbreviated month names

# Add grid for better readability
plt.grid(True, alpha=0.3)

# Add vertical line at election day (middle point)
# Loop through each date and add a vertical line
for date in election_dates:
    plt.axvline(x=date, color='gray', linestyle='--', alpha=0.8)

# Add a label for election dates (only once, to avoid repetition)
plt.axvline(x=election_dates[0], color='gray', linestyle='--', alpha=0.8, label='Election Day')



plt.legend()
plt.tight_layout()
plt.show()



# 3. Analyze the Data

## Analysis of CNY, EUR, and JPY Volatility

In [ ]:
# Create a new column called 'Lagged Currency Rate' for all of the seperated dataframes
df_china_daily['Lagged Currency Rate'] = df_china_daily['CNY'].shift(1)
df_euro_daily['Lagged Currency Rate'] = df_euro_daily['EUR'].shift(1)
df_japan_daily['Lagged Currency Rate'] = df_japan_daily['JPY'].shift(1)

# Add a column calculating the percent change with .pct_change() for all of the seperated dataframes
df_china_daily['Daily Rate Change'] = df_china_daily['CNY'].astype(float).pct_change()
df_euro_daily['Daily Rate Change'] = df_euro_daily['EUR'].astype(float).pct_change()
df_japan_daily['Daily Rate Change'] = df_japan_daily['JPY'].astype(float).pct_change()
 
# Add a column with the volatility of the exchanges rates on a monthly bases with a rolling window of 4 days using df_currency_rates
df_china_daily['Currency Volatility'] = df_china_daily['Daily Rate Change'].rolling(window=7).std()
df_euro_daily['Currency Volatility'] = df_euro_daily['Daily Rate Change'].rolling(window=7).std()
df_japan_daily['Currency Volatility'] = df_japan_daily['Daily Rate Change'].rolling(window=7).std()

In [ ]:
# Let's preview our currency data frames with the newly added columns
display(df_china_daily.head(10))
display(df_euro_daily.head(10))
display(df_japan_daily.head(10))

In [ ]:
# Let visualize the Volatility of CNY, EUR and JPY on one graph
plt.clf()
plt.figure(figsize=(18, 6))
plt.plot(df_china_daily['Currency Volatility'], label='CNY')
plt.plot(df_euro_daily['Currency Volatility'], label='EUR')
plt.plot(df_japan_daily['Currency Volatility'], label='JPY')
plt.legend()
plt.title('Currency Volatility')
plt.xlabel('Date')
plt.ylabel('Volatility')

# Add vertical line at election day (middle point)
# Loop through each date and add a vertical line
for date in election_dates:
    plt.axvline(x=date, color='gray', linestyle='--', alpha=0.8)

# Add a label for election dates (only once, to avoid repetition)
plt.axvline(x=election_dates[0], color='gray', linestyle='--', alpha=0.8, label='Election Day')
plt.show()

In [ ]:
election_windows

In [ ]:
election_windows[2000][0]

In [ ]:
election_windows[2000][1]

In [ ]:
# Let's Zoom in the election years looking at the graph closer - 15 days before and after election
# 2000 Election Year
plt.clf()
plt.figure(figsize=(18, 6))
plt.plot(df_china_daily['Currency Volatility'], label='CNY')
plt.plot(df_euro_daily['Currency Volatility'], label='EUR')
plt.plot(df_japan_daily['Currency Volatility'], label='JPY')
plt.legend()
plt.title('Currency Volatility - Election Year 2000')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.xlim(pd.to_datetime(election_windows[2000][0]), pd.to_datetime(election_windows[2000][1]))
plt.show()

In [ ]:
# 2004 Election Year
plt.clf()
plt.figure(figsize=(18, 6))
plt.plot(df_china_daily['Currency Volatility'], label='CNY')
plt.plot(df_euro_daily['Currency Volatility'], label='EUR')
plt.plot(df_japan_daily['Currency Volatility'], label='JPY')
plt.legend()
plt.title('Currency Volatility - Election Year 2004')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.xlim(pd.to_datetime(election_windows[2004][0]), pd.to_datetime(election_windows[2004][1]))
plt.show()

In [ ]:
# 2008 Election Year
plt.clf()
plt.figure(figsize=(18, 6))
plt.plot(df_china_daily['Currency Volatility'], label='CNY')
plt.plot(df_euro_daily['Currency Volatility'], label='EUR')
plt.plot(df_japan_daily['Currency Volatility'], label='JPY')
plt.legend()
plt.title('Currency Volatility - Election Year 2008')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.xlim(pd.to_datetime(election_windows[2008][0]), pd.to_datetime(election_windows[2008][1]))
plt.show()

In [ ]:
# 2012 Election Year
plt.clf()
plt.figure(figsize=(18, 6))
plt.plot(df_china_daily['Currency Volatility'], label='CNY')
plt.plot(df_euro_daily['Currency Volatility'], label='EUR')
plt.plot(df_japan_daily['Currency Volatility'], label='JPY')
plt.legend()
plt.title('Currency Volatility - Election Year 2012')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.xlim(pd.to_datetime(election_windows[2012][0]), pd.to_datetime(election_windows[2012][1]))
plt.show()

In [ ]:
# 2016 Election Year
plt.clf()
plt.figure(figsize=(18, 6))
plt.plot(df_china_daily['Currency Volatility'], label='CNY')
plt.plot(df_euro_daily['Currency Volatility'], label='EUR')
plt.plot(df_japan_daily['Currency Volatility'], label='JPY')
plt.legend()
plt.title('Currency Volatility - Election Year 2016')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.xlim(pd.to_datetime(election_windows[2016][0]), pd.to_datetime(election_windows[2016][1]))
plt.show()

In [ ]:
# 2020 Election Year
plt.clf()
plt.figure(figsize=(18, 6))
plt.plot(df_china_daily['Currency Volatility'], label='CNY')
plt.plot(df_euro_daily['Currency Volatility'], label='EUR')
plt.plot(df_japan_daily['Currency Volatility'], label='JPY')
plt.legend()
plt.title('Currency Volatility - Election Year 2020')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.xlim(pd.to_datetime(election_windows[2020][0]), pd.to_datetime(election_windows[2020][1]))
plt.show()

In [ ]:
# 2024 Election Year
plt.clf()
plt.figure(figsize=(18, 6))
plt.plot(df_china_daily['Currency Volatility'], label='CNY')
plt.plot(df_euro_daily['Currency Volatility'], label='EUR')
plt.plot(df_japan_daily['Currency Volatility'], label='JPY')
plt.legend()
plt.title('Currency Volatility - Election Year 2024')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.xlim(pd.to_datetime(election_windows[2024][0]), pd.to_datetime(election_windows[2024][1]))
plt.show()

In [521]:
# Let's investigate a certain date range to see how all three currencies perform around the 2024 election year. 
# We'll aslo choose a non-election year of 2022 so we have something to compare it to. 

# Let's set our variables for our start and end date for 2022 and 2024
start_date_2022 = '2022-06-22'
end_date_2022 = '2022-12-31'

start_date_2024 = '2024-06-22'
end_date_2024 = '2024-12-31'

# Slice the full data frame into the start and end dates for 2022 and 2024
df_china_daily_zoom_2022 = df_china_daily.loc[start_date_2022:end_date_2022]
df_china_daily_zoom_2024 = df_china_daily.loc[start_date_2024:end_date_2024]
df_euro_daily_zoom_2022 = df_euro_daily.loc[start_date_2022:end_date_2022]
df_euro_daily_zoom_2024 = df_euro_daily.loc[start_date_2024:end_date_2024]
df_japan_daily_zoom_2022 = df_japan_daily.loc[start_date_2022:end_date_2022]
df_japan_daily_zoom_2024 = df_japan_daily.loc[start_date_2024:end_date_2024]


In [ ]:
# Show the plots for CNY, EUR, and JPY Volatility
plt.clf()
df_china_daily_zoom_2022.plot(figsize=(12, 6), title="China Currency Volatility - 2022 (Non-Election Year)")
df_china_daily_zoom_2024.plot(figsize=(12, 6), title="China Currency Volatility - 2024 (Election Year)")
df_euro_daily_zoom_2022.plot(figsize=(12, 6), title="Euro Currency Volatility - 2022 (Non-Election Year)")
df_euro_daily_zoom_2024.plot(figsize=(12, 6), title="Euro Currency Volatility - 2024 (Election Year)")
df_japan_daily_zoom_2022.plot(figsize=(12, 6), title="Japan Currency Volatility - 2022 (Non-Election Year)")
df_japan_daily_zoom_2024.plot(figsize=(12, 6), title="Japan Currency Volatility - 2024 (Election Year)")
plt.show()

# Mine exchange rate data for seasonality

Mine the exchange rate data for predictable seasonal patterns

1. Group the daily exchange rate data to plot the average rate by day of the week. 
2. Group the daily exchange rate data to plot the average rate by week of the year. 


In [523]:
# Convert the CNY to the datatype float in df_china_daily
df_china_daily['CNY'] = df_china_daily['CNY'].astype(float)
df_china_daily['Lagged Currency Rate'] = df_china_daily['Lagged Currency Rate'].astype(float)

df_euro_daily['EUR'] = df_euro_daily['EUR'].astype(float)
df_euro_daily['Lagged Currency Rate'] = df_china_daily['Lagged Currency Rate'].astype(float)

df_japan_daily['JPY'] = df_japan_daily['JPY'].astype(float)
df_japan_daily['Lagged Currency Rate'] = df_china_daily['Lagged Currency Rate'].astype(float)

In [ ]:
# Group the daily exchange rate data to plot the average rate by month of year
median_montly_exchange_CNY = df_china_daily.groupby(df_china_daily.index.month).mean()
median_montly_exchange_EUR = df_euro_daily.groupby(df_euro_daily.index.month).mean()
median_montly_exchange_JPY = df_japan_daily.groupby(df_japan_daily.index.month).mean()

# Plot the volatility by month of year
plt.clf()
fig, ax = plt.subplots(figsize=(10, 6))
median_montly_exchange_CNY[‘Currency Volatility’].plot(ax=ax)
median_montly_exchange_EUR[‘Currency Volatility’].plot(ax=ax)
median_montly_exchange_JPY[‘Currency Volatility’].plot(ax=ax)

# Set custom x-ticks with month names
ax.set_xticks(range(1, 13))  # Set x-axis ticks to 1-12
ax.set_xticklabels(calendar.month_abbr[1:])  # Replace numbers with abbreviated month names
plt.xlabel(‘Month’)
plt.ylabel(‘Volatility’)
plt.legend([‘CNY’, ‘EUR’, ‘JPY’])
plt.title(‘Volatility of Exchange Rate by Month of the Year (2000 - 2024)’)
plt.show()

In [ ]:
# Group the daily exchange rate data to plot the average rate by week of the year
median_weekly_exchange_CNY = df_china_daily.groupby(df_china_daily.index.isocalendar().week).mean()
median_weekly_exchange_EUR = df_euro_daily.groupby(df_euro_daily.index.isocalendar().week).mean()
median_weekly_exchange_JPY = df_japan_daily.groupby(df_japan_daily.index.isocalendar().week).mean()

# Plot the weekly exchange rate data
plt.clf()
median_weekly_exchange_CNY['Currency Volatility'].plot(figsize=(10, 6), ylabel='Volatility')
median_weekly_exchange_JPY['Currency Volatility'].plot()
median_weekly_exchange_EUR['Currency Volatility'].plot()
plt.legend(['CNY', 'EUR', 'JPY'])
plt.xlabel('Week of the Year')
plt.title('Volatility of Exchange Rate by Week of the Year (2000 - 2024)')
plt.show()
plt.show()

In [ ]:
display(df_china_daily.head())
display(df_china_daily.tail())

In [ ]:
# Let's generate a dataframe where the index is the date from 2000-01-03 to 2025-01-17 with the column Party
df_party = pd.DataFrame(index=pd.date_range('2000-01-03', '2024-12-31'), columns=['Party'])

# name the index column as Date
df_party.index.name = 'Date'
df_party = df_party.reset_index()
df_party['Date'] = pd.to_datetime(df_party['Date'])

# preview the head and tail
display(df_party.head())
display(df_party.tail())


In [592]:
# let’s create function with multiple conditions that will assign the correct value
def assign_party(date):
    year = date.year
    if year >= 2000 and year < 2001: # Clinton (Democratic)
        return 1
    elif year >= 2001 and year < 2009: # Bush (Republic)
        return 0
    elif year >= 2009 and year < 2017: # Barack (Democratic)
        return 1
    elif year >= 2017 and year < 2021: # Trump (Republic)
        return 0
    elif year >= 2021 and year < 2025: # Biden (Democratic)
        return 1
    else:
        return None

In [593]:
# lets apply our function to the df_party 
df_party['Party'] = df_party['Date'].apply(assign_party) 

In [ ]:
# export df_party to a csv file to check
df_party.to_csv('Resources/df_party.csv', index=False)